# Ejercicio 1

## Importar librerías

In [21]:
from datasets import load_dataset

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

import mlflow

## EDA

He elegido [sms_spam](https://huggingface.co/datasets/ucirvine/sms_spam) como dataset porque es pequeño y manejable para ejecutar en local. Así es más fácil ejecutar más experimentos y poder compararlos con MLFlow.

In [2]:
# Cargamos dataset
ds = load_dataset("sms_spam")
ds

DatasetDict({
    train: Dataset({
        features: ['sms', 'label'],
        num_rows: 5574
    })
})

In [3]:
# Miramos que características tiene
ds["train"].features

{'sms': Value('string'), 'label': ClassLabel(names=['ham', 'spam'])}


Tenemos 5.574 filas que contienen el string del sms y una etiqueta para indicar si es spam ("1") o no ("0").

In [4]:
# Pasamos a pandas para hacer EDA
df = ds["train"].to_pandas()

# Cuántos hay 'ham' (0) y de 'spam' (1)
df["label"].value_counts(), (df["label"].value_counts(normalize=True) * 100).round(2)

(label
 0    4827
 1     747
 Name: count, dtype: int64,
 label
 0    86.6
 1    13.4
 Name: proportion, dtype: float64)

Aquí se ve que el dataset está desbalanceado. Es una proporción de 86.6% de ham y 13.4% de spam.

## Separación de train / test

In [5]:

# Dividir en train y test
X = df["sms"].astype(str)
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2, # por convención
    random_state=42, # para que sea reproducible
    stratify=y # para mantener proporción de clases
)

# Comprobamos proporciones
y_train.value_counts(normalize=True).round(3), y_test.value_counts(normalize=True).round(3)

(label
 0    0.866
 1    0.134
 Name: proportion, dtype: float64,
 label
 0    0.866
 1    0.134
 Name: proportion, dtype: float64)

Partimos el dataset de forma estratificada y fijamos un `random_state` para poder reproducir los pasos. Así conseguimos un dataset de train y un dataset de test que tienen la misma proporción de spam y ham que el dataset original.

## Preprocesamiento del texto

Usaremos `TfidfVectorizer` porque ya hace la tokenización y vectorización.

In [6]:
vectorizer = TfidfVectorizer(
    lowercase=True, # pasa todo el texto a minúsculas
    ngram_range=(1, 2),  # tamaño de token
    min_df=2, # ignora tokens en menos de 2 documentos
    max_df=0.95 # ignora tokens en más del 95% de documentos
)

vectorizer

,"input input: {'filename', 'file', 'content'}, default='content'- If `'filename'`, the sequence passed as an argument to fit is expected to be a list of filenames that need reading to fetch the raw content to analyze.- If `'file'`, the sequence items must have a 'read' method (file-like object) that is called to fetch the bytes in memory.- If `'content'`, the input is expected to be a sequence of items that can be of type string or byte.",'content'
,"encoding encoding: str, default='utf-8'If bytes or files are given to analyze, this encoding is used todecode.",'utf-8'
,"decode_error decode_error: {'strict', 'ignore', 'replace'}, default='strict'Instruction on what to do if a byte sequence is given to analyze thatcontains characters not of the given `encoding`. By default, it is'strict', meaning that a UnicodeDecodeError will be raised. Othervalues are 'ignore' and 'replace'.",'strict'
,"strip_accents strip_accents: {'ascii', 'unicode'} or callable, default=NoneRemove accents and perform other character normalizationduring the preprocessing step.'ascii' is a fast method that only works on characters that havea direct ASCII mapping.'unicode' is a slightly slower method that works on any characters.None (default) means no character normalization is performed.Both 'ascii' and 'unicode' use NFKD normalization from:func:`unicodedata.normalize`.",None
,"lowercase lowercase: bool, default=TrueConvert all characters to lowercase before tokenizing.",True
,"preprocessor preprocessor: callable, default=NoneOverride the preprocessing (string transformation) stage whilepreserving the tokenizing and n-grams generation steps.Only applies if ``analyzer`` is not callable.",None
,"tokenizer tokenizer: callable, default=NoneOverride the string tokenization step while preserving thepreprocessing and n-grams generation steps.Only applies if ``analyzer == 'word'``.",None
,"analyzer analyzer: {'word', 'char', 'char_wb'} or callable, default='word'Whether the feature should be made of word or character n-grams.Option 'char_wb' creates character n-grams only from text insideword boundaries; n-grams at the edges of words are padded with space.If a callable is passed it is used to extract the sequence of featuresout of the raw, unprocessed input... versionchanged:: 0.21 Since v0.21, if ``input`` is ``'filename'`` or ``'file'``, the data is first read from the file and then passed to the given callable analyzer.",'word'
,"stop_words stop_words: {'english'}, list, default=NoneIf a string, it is passed to _check_stop_list and the appropriate stoplist is returned. 'english' is currently the only supported stringvalue.There are several known issues with 'english' and you shouldconsider an alternative (see :ref:`stop_words`).If a list, that list is assumed to contain stop words, all of whichwill be removed from the resulting tokens.Only applies if ``analyzer == 'word'``.If None, no stop words will be used. In this case, setting `max_df`to a higher value, such as in the range (0.7, 1.0), can automatically detectand filter stop words based on intra corpus document frequency of terms.",None
,"token_pattern token_pattern: str, default=r""(?u)\\b\\w\\w+\\b""Regular expression denoting what constitutes a ""token"", only usedif ``analyzer == 'word'``. The default regexp selects tokens of 2or more alphanumeric characters (punctuation is completely ignoredand always treated as a token separator).If there is a capturing group in token_pattern then thecaptured group content, not the entire match, becomes the token.At most one capturing group is permitted.",'(?u)\\b\\w\\w+\\b'
,"ngram_range ngram_range: tuple (min_n, max_n), default=(1, 1)The lower and upper boundary of the range of n-values for differentn-grams to be extracted. All values of n such that min_n <= n <= max_nwill be used. For example an ``ngram_range`` of ``(1, 1)`` means onlyunigrams, ``(1, 2)`` means unigrams and bigrams, and ``(2, 2)`` meansonly bigrams.Only applies if ``analyzer`` is not callable.","(1

## Entrenamiento

### Modelo 1: CountVectorizer + MultinomialNB

In [7]:


# Pipeline
pipe_nb = Pipeline([
    ("vect", CountVectorizer(
        lowercase=True,
        ngram_range=(1, 2),
        min_df=2,
        max_df=0.95
    )),
    ("clf", MultinomialNB(alpha=1.0)),
])

In [8]:
# Entrenamiento
pipe_nb.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('vect', ...), ('clf', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"input input: {'filename', 'file', 'content'}, default='content'- If `'filename'`, the sequence passed as an argument to fit is expected to be a list of filenames that need reading to fetch the raw content to analyze.- If `'file'`, the sequence items must have a 'read' method (file-like object) that is called to fetch the bytes in memory.- If `'content'`, the input is expected to be a sequence of items that can be of type string or byte.",'content'
,"encoding encoding: str, default='utf-8'If bytes or files are given to analyze, this encoding is used todecode.",'utf-8'
,"decode_error decode_error: {'strict', 'ignore', 'replace'}, default='strict'Instruction on what to do if a byte sequence is given to analyze thatcontains characters not of the given `encoding`. By default, it is'strict', meaning that a UnicodeDecodeError will be raised. Othervalues are 'ignore' and 'replace'.",'strict'
,"strip_accents strip_accents: {'ascii', 'unicode'} or callable, default=NoneRemove accents and perform other character normalizationduring the preprocessing step.'ascii' is a fast method that only works on characters that havea direct ASCII mapping.'unicode' is a slightly slower method that works on any characters.None (default) means no character normalization is performed.Both 'ascii' and 'unicode' use NFKD normalization from:func:`unicodedata.normalize`.",None
,"lowercase lowercase: bool, default=TrueConvert all characters to lowercase before tokenizing.",True
,"preprocessor preprocessor: callable, default=NoneOverride the preprocessing (strip_accents and lowercase) stage whilepreserving the tokenizing and n-grams generation steps.Only applies if ``analyzer`` is not callable.",None
,"tokenizer tokenizer: callable, default=NoneOverride the string tokenization step while preserving thepreprocessing and n-grams generation steps.Only applies if ``analyzer == 'word'``.",None


In [9]:
# Predicción
y_pred_nb = pipe_nb.predict(X_test)

In [10]:
# Evaluación
accuracy_nb = accuracy_score(y_test, y_pred_nb)
precision_nb = precision_score(y_test, y_pred_nb, pos_label=1)
recall_nb = recall_score(y_test, y_pred_nb, pos_label=1)
f1_nb = f1_score(y_test, y_pred_nb, pos_label=1)

cm_nb = confusion_matrix(y_test, y_pred_nb)
report_nb = classification_report(y_test, y_pred_nb)

accuracy_nb, precision_nb, recall_nb, f1_nb, cm_nb

(0.9865470852017937,
 0.9785714285714285,
 0.9194630872483222,
 0.9480968858131488,
 array([[963,   3],
        [ 12, 137]]))

En este caso el accuracy no sirve por si sola para evaluar porque tenemos datasets desbalanceados (aprox. 86% ham y 14% spam).

En la matriz de confusión se ven:
* Pocos falsos positivos (3) --> La precision es de un 97,85%
* 12 falsos negativos de 149 que había en total --> El recall es de 91,94%

El F1 score que combina esas dos métricas sale en un 94,81%.

#### MLFlow

In [12]:
mlflow.set_experiment("sms spam classification")

with mlflow.start_run(run_name="sms-countvec-multinomial-nb"):
    # Logs de parámetros
    mlflow.log_param("vectorizer", "CountVectorizer")
    mlflow.log_param("lowercase", True)
    mlflow.log_param("ngram_range", "(1,2)")
    mlflow.log_param("min_df", 2)
    mlflow.log_param("max_df", 0.95)
    mlflow.log_param("model", "MultinomialNB")
    mlflow.log_param("alpha", 1.0)

    # Logs de métricas
    mlflow.log_metric("accuracy", accuracy_nb)
    mlflow.log_metric("spam_precision", precision_nb)
    mlflow.log_metric("spam_recall", recall_nb)
    mlflow.log_metric("spam_f1", f1_nb)

    # Logs de artefactos
    report_path = "classification_report_nb.txt"
    with open(report_path, "w", encoding="utf-8") as f:
        f.write(report_nb)
    mlflow.log_artifact(report_path)
    
    cm_path = "confusion_matrix_nb.csv"
    pd.DataFrame(
        cm_nb,
        index=["true_ham", "true_spam"],
        columns=["pred_ham", "pred_spam"]
    ).to_csv(cm_path)
    mlflow.log_artifact(cm_path)

    mlflow.sklearn.log_model(pipe_nb, "countvec_multinomial_nb")


2026/01/31 15:58:47 INFO mlflow.tracking.fluent: Experiment with name 'sms spam classification' does not exist. Creating a new experiment.
2026/01/31 15:58:47 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/Users/natalia/kc-ia-practica-mlops-llmops/practica/lib/python3.13/site-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


### Modelo 2. TF-ID + Logistic Regression

In [14]:
# Pipeline
pipe_lr = Pipeline([
    ("vect", TfidfVectorizer(
        lowercase=True,
        ngram_range=(1, 2),
        min_df=2,
        max_df=0.95
    )),
    ("clf", LogisticRegression(
        C=1.0,
        max_iter=1000,
        solver="liblinear"
    )),
])

In [15]:
# Entrenamiento
pipe_lr.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('vect', ...), ('clf', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"input input: {'filename', 'file', 'content'}, default='content'- If `'filename'`, the sequence passed as an argument to fit is expected to be a list of filenames that need reading to fetch the raw content to analyze.- If `'file'`, the sequence items must have a 'read' method (file-like object) that is called to fetch the bytes in memory.- If `'content'`, the input is expected to be a sequence of items that can be of type string or byte.",'content'
,"encoding encoding: str, default='utf-8'If bytes or files are given to analyze, this encoding is used todecode.",'utf-8'
,"decode_error decode_error: {'strict', 'ignore', 'replace'}, default='strict'Instruction on what to do if a byte sequence is given to analyze thatcontains characters not of the given `encoding`. By default, it is'strict', meaning that a UnicodeDecodeError will be raised. Othervalues are 'ignore' and 'replace'.",'strict'
,"strip_accents strip_accents: {'ascii', 'unicode'} or callable, default=NoneRemove accents and perform other character normalizationduring the preprocessing step.'ascii' is a fast method that only works on characters that havea direct ASCII mapping.'unicode' is a slightly slower method that works on any characters.None (default) means no character normalization is performed.Both 'ascii' and 'unicode' use NFKD normalization from:func:`unicodedata.normalize`.",None
,"lowercase lowercase: bool, default=TrueConvert all characters to lowercase before tokenizing.",True
,"preprocessor preprocessor: callable, default=NoneOverride the preprocessing (string transformation) stage whilepreserving the tokenizing and n-grams generation steps.Only applies if ``analyzer`` is not callable.",None
,"tokenizer tokenizer: callable, default=NoneOverride the string tokenization step while preserving thepreprocessing and n-grams generation steps.Only applies if ``analyzer == 'word'``.",None


In [16]:
# Predicción
y_pred_lr = pipe_lr.predict(X_test)

In [17]:
# Evaluación
accuracy_lr = accuracy_score(y_test, y_pred_lr)
precision_lr = precision_score(y_test, y_pred_lr, pos_label=1)
recall_lr = recall_score(y_test, y_pred_lr, pos_label=1)
f1_lr = f1_score(y_test, y_pred_lr, pos_label=1)

cm_lr = confusion_matrix(y_test, y_pred_lr)
report_lr = classification_report(y_test, y_pred_lr)

accuracy_lr, precision_lr, recall_lr, f1_lr, cm_lr

(0.9748878923766816,
 1.0,
 0.8120805369127517,
 0.8962962962962963,
 array([[966,   0],
        [ 28, 121]]))

En la matriz de confusión no aparece ningún falso positivo y la métrica precision sale como si fuera perfecta. Sin embargo, hay 28 falsos negativos y el recall es de 81,20%. Esto penaliza en el F1 score que resulta en 89,62%.

#### MLFlow

In [ ]:
mlflow.set_experiment("sms spam classification")

with mlflow.start_run(run_name="sms-tfidf-logreg"):
    # Logs de parámetros
    mlflow.log_param("vectorizer", "TfidfVectorizer")
    mlflow.log_param("lowercase", True)
    mlflow.log_param("ngram_range", "(1,2)")
    mlflow.log_param("min_df", 2)
    mlflow.log_param("max_df", 0.95)
    mlflow.log_param("model", "LogisticRegression")
    mlflow.log_param("C", 1.0)
    mlflow.log_param("max_iter", 1000)
    mlflow.log_param("solver", "liblinear")

    # Logs de métricas
    mlflow.log_metric("accuracy", accuracy_lr)
    mlflow.log_metric("spam_precision", precision_lr)
    mlflow.log_metric("spam_recall", recall_lr)
    mlflow.log_metric("spam_f1", f1_lr)
    
    # Logs de artefactos
    report_path = "classification_report_lr.txt"
    with open(report_path, "w", encoding="utf-8") as f:
        f.write(report_lr)
    mlflow.log_artifact(report_path)

    cm_path = "confusion_matrix_lr.csv"
    pd.DataFrame(
        cm_lr,
        index=["true_ham", "true_spam"],
        columns=["pred_ham", "pred_spam"]
    ).to_csv(cm_path)
    mlflow.log_artifact(cm_path)

    mlflow.sklearn.log_model(pipe_lr, "tfidf_logistic_regression")

2026/01/31 16:22:54 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/Users/natalia/kc-ia-practica-mlops-llmops/practica/lib/python3.13/site-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


### Modelo 3: TF-IDF + LinearSVC

In [22]:
# Pipeline
pipe_svc = Pipeline([
    ("vect", TfidfVectorizer(
        lowercase=True,
        ngram_range=(1, 2),
        min_df=2,
        max_df=0.95
    )),
    ("clf", LinearSVC(
        C=1.0
    )),
])

In [23]:
# Entrenamiento
pipe_svc.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('vect', ...), ('clf', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"input input: {'filename', 'file', 'content'}, default='content'- If `'filename'`, the sequence passed as an argument to fit is expected to be a list of filenames that need reading to fetch the raw content to analyze.- If `'file'`, the sequence items must have a 'read' method (file-like object) that is called to fetch the bytes in memory.- If `'content'`, the input is expected to be a sequence of items that can be of type string or byte.",'content'
,"encoding encoding: str, default='utf-8'If bytes or files are given to analyze, this encoding is used todecode.",'utf-8'
,"decode_error decode_error: {'strict', 'ignore', 'replace'}, default='strict'Instruction on what to do if a byte sequence is given to analyze thatcontains characters not of the given `encoding`. By default, it is'strict', meaning that a UnicodeDecodeError will be raised. Othervalues are 'ignore' and 'replace'.",'strict'
,"strip_accents strip_accents: {'ascii', 'unicode'} or callable, default=NoneRemove accents and perform other character normalizationduring the preprocessing step.'ascii' is a fast method that only works on characters that havea direct ASCII mapping.'unicode' is a slightly slower method that works on any characters.None (default) means no character normalization is performed.Both 'ascii' and 'unicode' use NFKD normalization from:func:`unicodedata.normalize`.",None
,"lowercase lowercase: bool, default=TrueConvert all characters to lowercase before tokenizing.",True
,"preprocessor preprocessor: callable, default=NoneOverride the preprocessing (string transformation) stage whilepreserving the tokenizing and n-grams generation steps.Only applies if ``analyzer`` is not callable.",None
,"tokenizer tokenizer: callable, default=NoneOverride the string tokenization step while preserving thepreprocessing and n-grams generation steps.Only applies if ``analyzer == 'word'``.",None


In [24]:
# Predicción
y_pred_svc = pipe_svc.predict(X_test)

In [25]:
# Evaluación
accuracy_svc = accuracy_score(y_test, y_pred_svc)
precision_svc = precision_score(y_test, y_pred_svc, pos_label=1)
recall_svc = recall_score(y_test, y_pred_svc, pos_label=1)
f1_svc = f1_score(y_test, y_pred_svc, pos_label=1)

cm_svc = confusion_matrix(y_test, y_pred_svc)
report_svc = classification_report(y_test, y_pred_svc)

accuracy_svc, precision_svc, recall_svc, f1_svc, cm_svc

(0.9874439461883409,
 0.9856115107913669,
 0.9194630872483222,
 0.9513888888888888,
 array([[964,   2],
        [ 12, 137]]))

En la matriz de confusión se ven: 2 falsos positivos y 12 falsos negativos. Hay una precisión y un recall altos.

#### MLFlow

In [26]:
mlflow.set_experiment("sms spam classification")

with mlflow.start_run(run_name="sms-tfidf-linearsvc"):
    # Logs de parámetros
    mlflow.log_param("vectorizer", "TfidfVectorizer")
    mlflow.log_param("lowercase", True)
    mlflow.log_param("ngram_range", "(1,2)")
    mlflow.log_param("min_df", 2)
    mlflow.log_param("max_df", 0.95)
    mlflow.log_param("model", "LinearSVC")
    mlflow.log_param("C", 1.0)

    # Logs de métricas
    mlflow.log_metric("accuracy", accuracy_svc)
    mlflow.log_metric("spam_precision", precision_svc)
    mlflow.log_metric("spam_recall", recall_svc)
    mlflow.log_metric("spam_f1", f1_svc)

    # Logs de artefactos
    report_path = "classification_report_svc.txt"
    with open(report_path, "w", encoding="utf-8") as f:
        f.write(report_svc)
    mlflow.log_artifact(report_path)

    cm_path = "confusion_matrix_svc.csv"
    pd.DataFrame(
        cm_svc,
        index=["true_ham", "true_spam"],
        columns=["pred_ham", "pred_spam"]
    ).to_csv(cm_path)
    mlflow.log_artifact(cm_path)

    mlflow.sklearn.log_model(pipe_svc, "tfidf_linear_svc")

2026/01/31 16:45:08 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/Users/natalia/kc-ia-practica-mlops-llmops/practica/lib/python3.13/site-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


## Comparación de modelos

Basado en las métricas lo que se puede observar es que:
* El algoritmo de Multinomial Naive Bayes detecta más spam pero tiene algún falso positivo.
* El algoritmo de regresión logística evita completamente los falsos positivos, pero deja pasar más spam.
* El algoritmo LinearSVC se ha quedado en medio entre los dos anteriores, tiene un falso positivo menos que NB y el mismo número de falsos negativos.

Como hablamos, al ser un dataset desbalanceado (aprox. 86% ham y 14% spam) la métrica de accuracy no nos sirve por si sola para evaluar. Si comparamos la F1-score, que es una métrica que combina precision y recall, el "ganador" sería LinearSVC con un 95,1%. Ligeramente por detrás estaría MultinomialNB con un 94,8% y Logistic Regression con un 89,6% porque le han penalizado mucho los falsos negativos (recall).

La elección de uno u otro dependerá de como de tolerantes queramos ser con los falsos positivos. Si se prioriza el equilibrio entre ambos, elegiría LinearSVC.

Hay que tener en cuenta que es un dataset extremadamente pequeño y la estimación del rendimiento no es muy fiable.